# Machine Learning

First, we should define models.  A *model* is a specification of a mathematical or probabilistic relationship that exists between different variables.  *Machine learning* is the creation and application of models that are learned from data.  In other contexts this may be known as *predictive modeling* or *data mining*.  Machine learning can be used to:

- Predict whether an email message is spam or not
- Predict whether a credit card transaction is fraudulent
- Predict which advertisement a shopper is most likely to click on
- Predict which football team is going to win the Super Bowl

There are two main types of machine learning; *supervised* and *unsupervised*.  Supervised modeling provides a data set that is labeled with the correct answer s to learn from.  Unsupervised modeling provides a data set with no labels.  There are also *semi-supervised* models in which some of the data are labeled, and *online* models, in which the model needs to continuously adjust to newly arriving data, but that won't be covered.

Among these models, you can either choose a parameterized model or learn the parameters that best fit the model from the data.

A common danger to machine learning is *overfitting*, where the model performs well on the given data, but underperforms during generalized utilization with newly introduced data.  Another scenario could be *underfitting*, where a model doesn't perform well even on the training data.

To train your model and avoid overfitting, a good way is to split your data.  Using two thirds to train your model, you can leave the rest to introduce as a test to ensure that the model isn't going to miss the newly introduced data set:

In [1]:
def split_data(data, prob):
    """split data into fractions [prob, 1 - prob]"""
    results = [], []
    for row in data:
        results[0 if random.random() < prob else 1].append(row)
    return results

Often, we'll have a matrix `x` of input variables and a vector `y` of output variables.  In that case, we need to make sure to put corresponding values together in either the training data or the test data:

In [4]:
def train_test_split(x, y, test_pct):
    data = list(zip(x, y))                        # pair corresponding values
    train, test = split_data(data, 1 - test_pct)  # split the dataset of pairs
    x_train, y_train = list(zip(*train))          # magical un-zip trick
    x_test, y_test = list(zip(*test))
    return x_train, x_test, y_train, y_test

So that you may be able to do something like:

```python
model = SomeKindOfModel()
    x_train, x_test, y_train, y_test = train_test_split(xs, ys, 0.33)
    model.train(x_train, y_train)
    performance = model.test(x_test, y_test)
```

If the model was overfit, then it should perform really poorly on the test data.  This can raise confidence that the model works but still may fail on a further larger data set.  Furthermore, the test set that your chosen model performs well on may in effect become a de facto training set.  So splitting your data into three sets - training data, validation data to confirm the efficacy of your chosen model, and test data to confirm that the model is not overfit.

When creating a model, the correctness of the data should be kept in mind.  When considering a binary judgement, decisions will lie in one of four categories:

- True positive: *the message is spam and we correctly predicted spam*
- False positive (Type 1 error): *the message is not spam but we predicted spam*
- False negative (Type 2 error): *the message is spam but we predicted not spam*
- True negative: *this message is not spam and we correctly predicted not spam*

Now, when it comes to correctness, the application of the model must produce sequiter conclusions.  Let's take the leukemia prediction example (people with the name "Luke" won't get leukemia; 98% accurate prediction model):

|name      |leukemia|no lukemia|total  |
|----------|:------:|:--------:|------:|
|"Luke"    |70      |4930      |5000   |
|not "Luke"|13930   |981070    |995000 |
|total     |14000   |986000    |1000000|

Let's use these numbers to compute various statistics about model performance:

In [7]:
def accuracy(tp, fp, fn, tn):
    correct = tp + tn
    total = tp + fp + fn + tn
    return correct / total

accuracy(70, 4930, 13930, 981070)

0.98114

This at first glance seems fairly impressive.  We should use precision to further analyse this model:

In [8]:
def precision(tp, fp, fn, tn):
    return tp / (tp + fp)

precision(70, 4930, 13930, 981070)

0.014

And let's use recall to measure what fraction of the positives our model identified:

In [10]:
def recall(tp, fp, fn, tn):
    return tp / (tp + fn)

recall(70, 4930, 13930, 981070)

0.005

`precision` and `recall` help us determine whether a model is terrible or not.  These functions are combined into the 
F1 score: 

In [11]:
def f1_score(tp, fp, fn, tn):
    p = precision(tp, fp, fn, tn)
    r = recall(tp, fp, fn, tn)

    return 2 * p * r / (p + r)

f1_score(70, 4930, 13930, 981070)

0.00736842105263158

This score is known as *harmonic mean*.  Typically the choice of a model involves a trade-off between `precision` and `recall` scores.  A model that predicts 'yes' when it is even a little bit confident will have a high `recall` score but a low `precision` score.  A model that predicts 'yes' only when it is absolutely certain will have a high `precision` score but a low `recall` score.  Its result ends up being a trade-off between false positives and false negatives.  Another way to think about overfitting is a trade-off between bias and variance.  The best solution will be more data.  The more data that you have, the harder it will be to overfit.

When your model lacks the required amount of features, it will be underfit.  Features are the inputs to your model.  Features are extracted from your data so that the model is informed of what it is looking for:

- Does the word Viagra appear in an email? *binary value*
- How many time does the letter d appear? *numeric value*
- What was the domain of the sender? *category from a discrete set of values*

These value types are what your features will be composed of.  Naive bayes is useful for yes or no type features.  Regression models are useful for numeric features.  Decision trees are useful for numeric or categorical data.